<a href="https://colab.research.google.com/github/rallyfranky/my-first-repo/blob/main/hate_speech_competition20221113.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
! pip install mecab-python3 unidic-lite
! pip install unidic
import MeCab
import re
from collections import Counter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 7.4 MB/s 
     |████████████████████████████████| 47.4 MB 992 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658837 sha256=e6f4460cf5f7b2b78e54de6ad6cd99f45fd68590489d45d346994776678c610c
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7425 sha256=3710207e32e8888d794ee58beb52b2907db6ce85d2134e89ac2d9a40f22e701f
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e16dfd0a12411ec6b1122
Successfully built unidic


In [3]:
!python -m unidic download

download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:17<00:00, 29.5MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir


In [4]:
import MeCab

text = "私は機械学習が好きです。"

m=MeCab.Tagger("-Owakati")
text_segmented = m.parse(text)
print(text_segmented)

私 は 機械 学習 が 好き です 。 



In [5]:
!pip install transformers==2.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 635 kB 6.3 MB/s 
     |████████████████████████████████| 5.6 MB 49.4 MB/s 
     |████████████████████████████████| 880 kB 59.6 MB/s 
     |████████████████████████████████| 1.3 MB 74.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=299181d19c0f00a10a890a7b14ada472648864e9cd0103ed62d80c1a1a1ce86b
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [6]:
#ber
import torch
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

In [8]:
tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

In [9]:
tknz.tokenize('私は中国と日本のハーフである')

['私', 'は', '中国', 'と', '日本', 'の', 'ハーフ', 'で', 'ある']

In [10]:
tknz.encode('私は中国と日本のハーフである')

[2, 1325, 9, 780, 13, 91, 5, 9241, 12, 31, 3]

前処理

In [11]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
train.head()
train.tail()

,id,source,text,label
5251,42b5f86b0,news4vip,車じゃなくてもよくない？\nケーブル網を張り巡らせてリフトみたいなのを付けるとか,0
5252,402ce15d9,livejupiter,左やぞ？\n出すなら下水流,0
5253,0739a9fcb,newsplus,日本によって、けんけんガクガクの議論を持たらされた韓国は被害者\n\n日本人がしっかり考えな...,0
5254,d496c7dc8,news4vip,ゴムボート買って、沖まで漕いで行ったら？\n魚の血を塗っておけばサメが食べてくれるよ,0
5255,d1838f4f0,news4vip,男は美化してないだろ？普通にエロい部分や格好悪い所も描かれてる\n女の方は主人公美少女しかい...,0


In [ ]:
X = train.iloc[:,2]
y = train.iloc[:,3]

In [ ]:
tokeData=[]
select = ["動詞","形容詞","名詞","代名詞","助詞","助動詞","副詞","接続詞","感動詞"]
mecab = MeCab.Tagger('-Owakati')
mecab.parse('')
#分かち書きと品詞の種類（select）に合わせた絞り込みの関数作成。
def wakati_text(text):
    # 分けてノードごとにする
    node = mecab.parseToNode(text)
    terms = []

    while node:

        # 単語
        term = node.surface
        # 品詞
        pos = node.feature.split(',')[0]
        # もし品詞が条件と一致してたら
        if pos in select:
            terms.append(term)
        node = node.next
    # スペースを入れながら結合する。
    text_result = ' '.join(terms)
    return text_result

#行数分関数を実行し、tokeDataのリストに格納
for i in range(len(X)):
  tmp = wakati_text(X[i])
  tokeData.append(tmp)

In [ ]:
tokeData

['に 相手 さ れ て ない 人 と の 関係 なんて に する もの か',
 '最近 は アヘ アヘ QS マン や ない ｲｲ 1 0 Ａ ｲｸﾅｲ',
 '日本 と し て 生まれ て も 無能 な 学歴 って 分かっ たら 日本 の 権利 剥奪 し て 追放す べき やろ 甘える な',
 'よくよく 思え ば 川上 は 配布 に し たら とんでも なく 有能 だ よ な ガチャ から 引い たら 圧倒 歓喜 レベル や で',
 '押井 は 原作 レイプ の 専門 だ から 原作 マンガ の 真意 を 誤解 さ せる こと に 関し て は プロ だ が それ 以外 に は 何 も 取り柄 が ない',
 'マジレス する と 野党 に 任せ て ドイツ し た 場合 税金 で 難民 を 養う こと に なる ね 生活 保護 どこ の 話 で は ない 金 の 話 を する なら 視野 を 広く せんと な ｗ',
 'だ から 退治 な わけ 殺さ れ た と か 無実 の 人間 な 言い する な 退治 さ れ た だけ だろう が',
 'D 柳 の 明治 と 京田 の 日大 を 応援 する 予定 両方 シード に 来 て 欲しい なあ',
 'キャラクター は これ と いっ て な もの は あり ませ ん なに か 物事 を とにかく キャラクター に する な 感じ です イメージ も 人 それぞれ です',
 '都内 の 幹線 道路 の 一角 に い た 百 円 で 醤油 の 効い た 海苔巻き 餅 を 売っ て くれ た オジ あれ は 旨かっ た',
 '女 は 精子 が なく て も 男 は 卵子 が なく て も 産める と は おもう 段階 で も 倫理 が 人間 の 進歩 を 疎外 し てる の か な',
 '付加 機能 が なに も ない 一番 安い 機種 だ ね 数字 の ボタン 押し て 電話 掛ける だけ の ヤツ',
 '神 に 愛 を 捧げる 人生 で 良い 転機 が 訪れ た の は 大体 私 の 中 で は 神社 や 神道 が 絡ん で くる とき',
 'キラ 付け やっ て ない の よ',
 'だ けど 予定 ある と か 気 が 乗ら ない と か って 普通 に 断れ ば いい けど 今月 は 

In [ ]:
df = pd.DataFrame(tokeData)

In [ ]:
df.head()

,0
0,に 相手 さ れ て ない 人 と の 関係 なんて に する もの か
1,最近 は アヘ アヘ QS マン や ない ｲｲ 1 0 Ａ ｲｸﾅｲ
2,日本 と し て 生まれ て も 無能 な 学歴 って 分かっ たら 日本 の 権利 剥奪 ...
3,よくよく 思え ば 川上 は 配布 に し たら とんでも なく 有能 だ よ な ガチャ ...
4,押井 は 原作 レイプ の 専門 だ から 原作 マンガ の 真意 を 誤解 さ せる こと...


In [ ]:
#doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
sentence=[]
for text in df[0]:
  text_list = text.split(' ')
  sentence.append(text_list)

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentence)]
model = Doc2Vec(documents, vector_size=100, window=3, min_count=1, workers=4,dm=1)

In [ ]:
model.docvecs.most_similar(2)

[(729, 0.9843289852142334),
 (4100, 0.9843239784240723),
 (2022, 0.9840526580810547),
 (4433, 0.9840448498725891),
 (3576, 0.9840245842933655),
 (1913, 0.9839881658554077),
 (4610, 0.9839781522750854),
 (2755, 0.9838059544563293),
 (2567, 0.9837418794631958),
 (1094, 0.983696699142456)]

In [ ]:
train.iloc[1094]

id                                                90f7bb92c
source                                             news4vip
text      課金自慢があんまりできないとなると知識自慢、つぎ込んだ時間自慢、キャラ愛自慢、ゲームのめり込...
label                                                     0
Name: 1094, dtype: object

In [ ]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  #実際の分散表現

(14281, 100)
[[-6.2491959e-01 -1.1752551e+00  1.2504741e+00 ...  1.0598227e+00
  -3.3731955e-01 -1.5796277e-01]
 [-6.1946332e-01 -1.1531609e+00  1.1936077e+00 ...  1.0546248e+00
  -3.4035000e-01 -1.6819851e-01]
 [-6.1001956e-01 -1.1727005e+00  1.2132437e+00 ...  1.0499830e+00
  -3.3319962e-01 -1.7635259e-01]
 ...
 [-5.5114877e-05 -9.1791395e-03  4.7226432e-03 ...  2.2717281e-03
  -5.3269598e-03 -3.7560496e-03]
 [-1.6804906e-03 -5.6141578e-03  6.8790545e-03 ...  7.1700383e-03
  -3.8794870e-03 -7.0752786e-04]
 [-2.2554027e-03 -5.9192451e-03  7.5944532e-03 ...  7.2414461e-03
  -3.5887687e-03 -1.4120394e-03]]


In [ ]:
train.where(train['label']==1)

,id,source,text,label
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1.0
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
5251,NaN,NaN,NaN,NaN
5252,NaN,NaN,NaN,NaN
5253,NaN,NaN,NaN,NaN
5254,NaN,NaN,NaN,NaN


In [ ]:
X_vec = pd.DataFrame(tokeData)
df = X_vec.merge(y)
df.head()

MergeError: ignored

In [ ]:
#まずはl2
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_vec_std = std.fit_transform(X_vec)
lr = LogisticRegression(C=10,penalty='l2',random_state=1)

In [ ]:
#ホールドアウト
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_vec_std, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=lr,X=X_train,y=y_train,cv=10,n_jobs=1)

In [ ]:
print('accuracy is %s' % scores)

In [ ]:
print('accuracy is %.3f +- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
y_pred = lr.predict(X_val)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, f1_score

print('Precision is %.3f' % precision_score(y_true=y_val, y_pred=y_pred))
print('Recall is %.3f' % recall_score(y_true=y_val, y_pred=y_pred))
print('f1 is %.3f' % f1_score(y_true=y_val, y_pred=y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
confmat = confusion_matrix(y_true=y_val, y_pred=y_pred)
print(confmat)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_wakati, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)

#パイプライン
param_grid = [{'vect__ngram_range':[(1,1)],'clf__penalty':['l1','l2'],'clf__C':[1.0,10.0,100.0]},
              {'vect__ngram_range':[(1,1)],'vect__use_idf':[False],'vect__norm':[None],'clf__penalty':['l1','l2'],'clf__C':[1.0,10.0,100.0]}]
lr_tfidfvec = Pipeline([('vect',tfidfvec),('clf',LogisticRegression(random_state=0,solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidfvec,param_grid,scoring='f1',cv=5,verbose=2,n_jobs=1)
gs_lr_tfidf.fit(X_train,y_train)

In [ ]:
print('paramater: %s' % gs_lr_tfidf.best_params_)

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('test f1 is: %.3f' % clf.score(X_val, y_val))

In [ ]:
y_pred = gs_lr_tfidf.predict(X_val)

In [ ]:
print('Precision is %.3f' % precision_score(y_true=y_val, y_pred=y_pred))
print('Recall is %.3f' % recall_score(y_true=y_val, y_pred=y_pred))
print('f1 is %.3f' % f1_score(y_true=y_val, y_pred=y_pred))

In [ ]:
confmat = confusion_matrix(y_true=y_val, y_pred=y_pred)
print(confmat)